In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('news.csv')
df = df[['text','label']]
df.dropna(inplace=True)

In [3]:
df.head()

,text,label
0,WASHINGTON (Reuters) - Two U.S. senators from ...,0
1,BAGHDAD (Reuters) - The Iraqi government gave ...,0
2,LONDON (Reuters) - British minister Michael Go...,0
3,"CHARLOTTE, N.C. (Reuters) - Republican preside...",0
4,"Following the situation in Charlottesville, Tr...",1


In [4]:
df.shape

(44898, 2)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

texts = df['text'].values
labels = df['label'].values

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)

import pickle
with open('tokenizer.pkl','wb') as f:
    pickle.dump(tokenizer,f)

sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences,maxlen=500)

x_train,x_test,y_train,y_test = train_test_split(padded,labels,test_size=0.2,random_state=42)

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential

model = Sequential([
    tf.keras.layers.Embedding(10000,64,input_length=500),
    tf.keras.layers.LSTM(64,dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [12]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.fit(x_train,y_train,epochs=4,validation_data=(x_test,y_test))

Epoch 1/4
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 357s 313ms/step - accuracy: 0.9406 - loss: 0.1863 - val_accuracy: 0.9870 - val_loss: 0.0517
Epoch 2/4
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 382s 340ms/step - accuracy: 0.9689 - loss: 0.0818 - val_accuracy: 0.9847 - val_loss: 0.0525
Epoch 3/4
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 373s 332ms/step - accuracy: 0.9850 - loss: 0.0452 - val_accuracy: 0.9388 - val_loss: 0.1555
Epoch 4/4
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 359s 320ms/step - accuracy: 0.9836 - loss: 0.0439 - val_accuracy: 0.9874 - val_loss: 0.0456


In [15]:
model.save('lstm_fake_news_model.h5')